In [ ]:
# =================================================================================
# ||  Final, Optimized: OpenStreetMap Multi-Key Data Extractor                 ||
# ||  Features: Place Name Search, Interactive Map AOI, and GeoJSON Upload     ||
# =================================================================================

# 1. Install required libraries if you haven't already
# !pip install osmnx geopandas ipywidgets geopy geemap ipyleaflet

import osmnx as ox
import geopandas as gpd
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from geopy.geocoders import Nominatim
from google.colab import drive
import os
import pandas as pd
import geemap
import ee
from ipyleaflet import DrawControl
import io

# Authenticate and initialize Earth Engine (for geemap background)
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='PROJECT ID')

# 2. Initialize the geolocator for location searches
geolocator = Nominatim(user_agent="osm_downloader_optimized")

# 3. Define the list of top-level OSM keys (features)
osm_keys = [
    'aerialway', 'aeroway', 'amenity', 'barrier', 'boundary', 'building', 'craft',
    'emergency', 'geological', 'healthcare', 'highway', 'historic', 'landuse',
    'leisure', 'man_made', 'military', 'natural', 'office', 'place', 'power',
    'public_transport', 'railway', 'route', 'shop', 'telecom', 'tourism', 'water',
    'waterway'
]

# 4. Global Variables for State Management
locations_to_download = [] # Stores tuples of (Location Name, Type)
uploaded_files = {}      # Stores uploaded file content: {filename: content}
m = None                 # Placeholder for the geemap.Map instance
output_text = widgets.Output() # For status messages
map_output_widget = widgets.Output() # For map-specific messages

# 5. Create the User Interface Widgets

# --- Key Selection Widgets ---
key_select = widgets.SelectMultiple(
    options=osm_keys,
    description='Keys:',
    disabled=False,
    layout=widgets.Layout(height='120px')
)
select_all_keys_checkbox = widgets.Checkbox(value=False, description='Select All Keys')

# --- Location Input Widgets ---
location_input = widgets.Text(description='Location:', placeholder='Type a city name...')
location_suggestions_output = widgets.Output()
selected_locations_output = widgets.VBox()

# --- AOI/Map/File Widgets ---
start_map_button = widgets.Button(description='Define on Map', icon='map', button_style='info')
file_upload_widget = widgets.FileUpload(
    accept='.geojson',
    multiple=False,
    description='Upload GeoJSON'
)
add_bbox_button = widgets.Button(description='Add Drawn BBox', disabled=True, button_style='success', icon='check')

# --- Control Widgets ---
folder_input = widgets.Text(value='OSM_Data_Extractor', description='Drive Folder:')
run_button = widgets.Button(description="Run Query & Save", button_style='danger', icon='download')

# 6. Core Functions (UI and Helpers)

def update_selected_locations_display():
    """Refreshes the list of selected locations shown in the UI."""
    children = []
    if not locations_to_download:
        children.append(widgets.Label("No locations selected."))
    else:
        for item in locations_to_download:
            loc_name, loc_type = item
            display_name = f"📍 [{loc_type}] {loc_name}"
            if loc_type == 'FileUpload':
                display_name = f"📂 [File] {loc_name}"

            remove_button = widgets.Button(description='Remove', button_style='danger', layout=widgets.Layout(width='auto'))
            location_label = widgets.Label(display_name)

            def on_remove_click(b, location_to_remove=item):
                if location_to_remove in locations_to_download:
                    locations_to_download.remove(location_to_remove)
                    if location_to_remove[1] == 'FileUpload':
                        uploaded_files.pop(location_to_remove[0], None)
                    update_selected_locations_display()
            remove_button.on_click(on_remove_click)
            children.append(widgets.HBox([location_label, remove_button]))
    selected_locations_output.children = children

def on_file_upload(change):
    """Handles the GeoJSON file upload event."""
    with output_text:
        try:
            uploaded_file_dict = change.new
            if not uploaded_file_dict: return
            filename, file_info = list(uploaded_file_dict.items())[0]
            uploaded_files[filename] = file_info['content']
            new_location_tuple = (filename, 'FileUpload')
            if new_location_tuple not in locations_to_download:
                locations_to_download.append(new_location_tuple)
                update_selected_locations_display()
        except Exception as e:
            print(f"❌ Error during file upload: {e}")
        finally:
            file_upload_widget.value.clear()
            file_upload_widget._counter = 0

def on_location_input_change(change):
    """Provides real-time location suggestions."""
    query = change.new
    if len(query) < 3:
        with location_suggestions_output: clear_output()
        return
    try:
        locations = geolocator.geocode(query, exactly_one=False, limit=5)
        with location_suggestions_output:
            clear_output()
            if not locations:
                print("No suggestions found.")
                return
            for loc in locations:
                button = widgets.Button(description=loc.address, button_style='info', layout={'width': '100%'})
                def on_button_click(b):
                    new_loc = (b.description, 'Place')
                    if new_loc not in locations_to_download:
                        locations_to_download.append(new_loc)
                        update_selected_locations_display()
                    location_input.value = ''
                    clear_output(wait=True)
                button.on_click(on_button_click)
                display(button)
    except Exception as e:
        with location_suggestions_output:
            clear_output()
            print(f"Error fetching suggestions: {e}")

def initialize_map():
    """Initializes the geemap instance for drawing an AOI."""
    global m
    if m is None:
        m = geemap.Map(center=[29.8543, 77.8880], zoom=12, layout=widgets.Layout(height='500px')) # Centered on Roorkee
        dc = DrawControl(rectangle={'shapeOptions': {'color': '#0000FF'}}, polygon={}, circle={}, polyline={}, marker={})
        def handle_draw(target, action, geo_json):
            if action == 'created' and geo_json['geometry']['type'] == 'Polygon':
                coords = geo_json['geometry']['coordinates'][0]
                min_lon, max_lon = min(p[0] for p in coords), max(p[0] for p in coords)
                min_lat, max_lat = min(p[1] for p in coords), max(p[1] for p in coords)
                bbox_str = f'{max_lat},{min_lat},{max_lon},{min_lon}'
                add_bbox_button.disabled = False
                add_bbox_button.bbox_value = bbox_str
                with map_output_widget:
                    clear_output(wait=True)
                    print(f"BBox drawn. Click 'Add Drawn BBox' to add it.")
            elif action == 'deleted':
                add_bbox_button.disabled = True
                with map_output_widget:
                    clear_output(wait=True)
                    print("Area deleted. Draw a new rectangle.")
        dc.on_draw(handle_draw)
        m.add(dc)

def on_start_map_click(b):
    """Switches the UI to the map view."""
    initialize_map()
    main_ui_box.layout.display = 'none'
    map_ui_box.layout.display = 'flex'
    display(m)
    add_bbox_button.disabled = True
    with map_output_widget:
        clear_output(wait=True)
        print("Use the rectangle tool (▭) to draw an Area of Interest.")

def on_add_bbox_click(b):
    """Adds the drawn bounding box to the list and returns to main UI."""
    new_location = (b.bbox_value, 'BBox')
    if new_location not in locations_to_download:
        locations_to_download.append(new_location)
        update_selected_locations_display()
    map_ui_box.layout.display = 'none'
    main_ui_box.layout.display = 'flex'
    with map_output_widget: clear_output()
    add_bbox_button.disabled = True
    if m and m.draw_control: m.draw_control.clear()

# 7. Main Download Function
def on_run_button_click(b):
    """Executes the download process with a single, combined query per location."""
    with output_text:
        clear_output()
        keys_to_download = osm_keys if select_all_keys_checkbox.value else list(key_select.value)

        if not locations_to_download or not keys_to_download:
            print("❌ Please select at least one location/AOI AND at least one OSM Key.")
            return

        print("Mounting Google Drive...")
        try:
            drive.mount('/content/drive', force_remount=True)
            base_save_path = f'/content/drive/My Drive/{folder_input.value}'
            os.makedirs(base_save_path, exist_ok=True)
        except Exception as e:
            print(f"❌ Error mounting drive: {e}")
            return

        # --- OPTIMIZATION: Combine all keys into a single dictionary ---
        # This creates a dictionary like {'building': True, 'highway': True, ...}
        # OSMnx will query for features that have ANY of these keys.
        tags = {key: True for key in keys_to_download}

        for location_query, loc_type in locations_to_download:
            print(f"\n--- Processing {loc_type}: {location_query} ---")
            print(f"  > Sending ONE combined query for {len(keys_to_download)} keys...")

            try:
                gdf = gpd.GeoDataFrame() # Initialize an empty GeoDataFrame

                # --- Make a SINGLE API call per location ---
                if loc_type == 'Place':
                    gdf = ox.features_from_place(location_query, tags=tags)
                    location_clean = location_query.split(',')[0].replace(' ', '_')[:50]
                elif loc_type == 'BBox':
                    north, south, east, west = map(float, location_query.split(','))
                    bbox = (north, south, east, west)
                    gdf = ox.features_from_bbox(bbox, tags=tags)
                    location_clean = f"BBox_{north:.2f}_{south:.2f}_{east:.2f}_{west:.2f}"
                elif loc_type == 'FileUpload':
                    geojson_content = uploaded_files[location_query].decode('utf-8')
                    aoi_gdf = gpd.read_file(io.StringIO(geojson_content))
                    aoi_polygon = aoi_gdf['geometry'].iloc[0]
                    gdf = ox.features_from_polygon(aoi_polygon, tags=tags)
                    location_clean = os.path.splitext(location_query)[0].replace(' ', '_')

                if not gdf.empty:
                    print(f"  > Found {len(gdf)} features in total.")
                    key_prefix = "ALL_KEYS" if select_all_keys_checkbox.value else "MULTIPLE_KEYS" if len(keys_to_download) > 1 else keys_to_download[0]
                    filename = f"{key_prefix}_{location_clean}.geojson"
                    full_path = os.path.join(base_save_path, filename)

                    gdf.to_file(full_path, driver='GeoJSON')
                    print(f"\n✅ Success! Saved to: {full_path}")
                else:
                    print(f"\n⚠️ No features found for the selected keys in {location_query}.")

            except Exception as e:
                print(f"  > ❌ An error occurred during the query for {location_query}: {e}")
                continue

# 8. Attach Functions to Widget Events
def on_select_all_keys_change(change):
    key_select.value = osm_keys if change.new else ()
    key_select.disabled = change.new
select_all_keys_checkbox.observe(on_select_all_keys_change, names='value')
location_input.observe(on_location_input_change, names='value')
run_button.on_click(on_run_button_click)
start_map_button.on_click(on_start_map_click)
add_bbox_button.on_click(on_add_bbox_click)
file_upload_widget.observe(on_file_upload, names='value')

# 9. Define UI Layouts and Display
key_selection_box = widgets.VBox([widgets.Label("1. Select OSM Key(s) to Download"), select_all_keys_checkbox, key_select])
location_selection_box = widgets.VBox([
    widgets.Label("2. Define Download Area(s)"),
    widgets.HTML("<b>Option A:</b> Search by place name."), location_input, location_suggestions_output,
    widgets.HTML("<hr><b>Option B:</b> Draw on map  OR  <b>Option C:</b> Upload file."),
    widgets.HBox([start_map_button, file_upload_widget]),
    widgets.HTML("<hr>"),
    widgets.Label("Selected Areas to Download:"), selected_locations_output,
])
control_box = widgets.VBox([widgets.Label("3. Configure Save Folder & Run"), folder_input, run_button])
main_ui_box = widgets.VBox([
    widgets.HTML("<h2>OSM Multi-Feature Data Extractor</h2>"),
    key_selection_box, location_selection_box, control_box, output_text
], layout=widgets.Layout(display='flex', flex_flow='column', align_items='stretch'))
map_ui_box = widgets.VBox([
    widgets.HTML("<h3>Draw Bounding Box on Map</h3>"), map_output_widget,
    widgets.HBox([add_bbox_button, widgets.Button(description='<< Back', on_click=lambda b: (setattr(main_ui_box.layout, 'display', 'flex'), setattr(map_ui_box.layout, 'display', 'none')), button_style='warning')])
], layout=widgets.Layout(display='none'))

# Initial Display Setup
display(main_ui_box)
display(map_ui_box)
update_selected_locations_display()